# Differential cross section fitting using BRICK 
## $^{12}\rm{C}(^{12}C, \alpha_{0})^{20}\rm{Ne}$ 

## Steps
* Construct a Python object that enables the calling of AZURE2 with an arbitrary input vector $\theta$.
* Set up priors for each of the sampled parameters and a corresponding prior ($\ln\Pi$) function.
* Construct a global function that reads a SRIM file, input energy, and return the -dE/dx.
* Construct the yield function.
* Set up a likelihood ($\ln\mathcal{L}$) function.
* Construct a posterior ($\ln\mathcal{P}$) function from the likelihood and prior functions.
* Prepare the MCMC walker and sample the posterior ($\ln\mathcal{P}$) function with emcee.
* Preview the results.

In [ ]:
import os
import emcee
import corner

import numpy as np
from scipy.interpolate import interp1d
from scipy.integrate import quad, quadrature
# import seaborn as sns
import matplotlib.pyplot as plt

import re
from tqdm import tqdm
from scipy import stats
from brick.azr import AZR
from multiprocess import Pool

### Step 1
* Construct a Python object that enables the calling of AZURE2 with an arbitrary input vector $\theta$.

In [ ]:
# Restrict processes to one thread only. Good for AZURE2
os.environ['OMP_NUM_THREADS'] = '1'

# Construct an object that calls AZURE2 and takes input 'a0.arz'
azr = AZR('a0.arz')

# How many parameters are going to be sampled?
nd = azr.config.nd + 1
print(f'There are {nd} sampled parameters.\n')

# Get the initial input parameters
# theta0 = azr.config.get_input_values()

# Read initial parameters from previously fitted results.
theta0 = []
theta0_mask = []
# initial_params_file = "best-emcee.txt"
# with open(initial_params_file, "r") as file:
#     for line in file:
#         # Split the line by ":"
#         parts = line.strip().split(":")
#         # Extract the numeric expression (second part after ":")
#         expression = parts[1].strip()
#         # Split the expression into terms
#         # Example: "9.07146 + 3.37034 - 3.77688" becomes ["9.07146", "3.37034", "3.77688"]
#         expression = expression.replace("+", " ").replace("-", " ").split()
#         # Convert to floats
#         base_value = float(expression[0])  # First number (base value)
#         addition_value = float(expression[1])  # Second number (value after "+")
#         subtraction_value = float(expression[2])  # Third number (value after "-")
#         # Append the base value to column_2_list
#         theta0.append(base_value)
#         # Find the max between addition_value and subtraction_value
#         theta0_mask.append(100.*max(addition_value, subtraction_value))

for i in range(nd-1):
    theta0.append(1000)
theta0.append(1E10)

print(f'The initial input parameters are:\n {theta0}\n')
print(f'The initial input parameters width are:\n {theta0_mask}\n')

# What labels have been assigned to those parameters?
labels = azr.config.labels
labels.append('$N_{v}$')
print(f'The labels of the input parameters are:\n {labels}\n')

In [ ]:
# Read in the data files
data_files_list = os.listdir('data/')
data_files_list.sort()
print(data_files_list)
num_energies = len(data_files_list)
energies = [float(filename.replace('.txt', '')) for filename in data_files_list]
energies = np.array(energies)
data = []
angle_lab = []
yield_lab = []
yield_err = []
for i in range(num_energies):
    temp = np.loadtxt("data/"+data_files_list[i])
    temp[:,2]=temp[:,2]*1E-15 # Scale factor of the yield
    temp[:,3]=temp[:,3]*1E-15 # Scale factor of the yield's error
    data.append(temp)
    angle_lab.append(temp[:,1])
    yield_lab.append(temp[:,2])
    yield_err.append(temp[:,3])
num_angles = len(angle_lab[0])
angle_lab = np.asarray(angle_lab, dtype='float32').T
yield_lab = np.asarray(yield_lab, dtype='float32').T
yield_err = np.asarray(yield_err, dtype='float32').T
print('shape of yield_lab is', yield_lab.shape)

# Take a look at the data
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
for i in range(num_energies):
    ax.errorbar(angle_lab[:,i], yield_lab[:,i], yerr=yield_err[:,i], linestyle='', capsize=2, fmt="o", label=data_files_list[i])
    # ax.scatter(angle_lab[i], yield_lab[i])
ax.legend()
ax.set_xlim(100, 140)
ax.set_xlabel(r'$\theta$ (deg, lab)')
ax.set_ylabel(r'Yields (arb. units)')

### Step 2
* Set up priors for each of the sampled parameters and a corresponding prior ($\ln\Pi$) function.

In [ ]:
# priors = []
# for i in range(nd-1):
#    if (i == 0) or (i==4) or (i==8) or (i==12) or (i==16) or (i==20) or (i==25) or (i==30) or (i==35) or (i==40) or (i==45) or (i==50) or (i==55) or (i==60) or (i==65):
#       priors.append(stats.norm(theta0[i], 0.1*theta0[i]/2.355))
#    else:
#       priors.append(stats.uniform(0, 1E5))
# priors.append(stats.uniform(1E4,1E6))

priors = []
for i in range(len(theta0)-1):
    priors.append(stats.uniform(0,1E6))
    # priors.append(stats.norm(theta0[i], theta0_mask[i]))
priors.append(stats.uniform(1E9,1E11))

# mask = np.array([0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8]) # Widths of the Gaussians can be different.
# priors = [stats.norm(mu, abs(mu) * mask[i]) for i, mu in enumerate(theta0)]

def lnPi(theta):
    return np.sum([pi.logpdf(t) for (pi, t) in zip(priors, theta)])

### Step 3
* Construct a global function that reads a SRIM file, input energy, and return the -dE/dx.

In [ ]:
# Read in the SRIM table
srim_file = 'srim/CarbonInCarbon.txt'
srim_table = []
with open(srim_file, 'r') as file:
    for line in file:
        cols = line.split()
        srim_table.append((float(cols[0]), float(cols[1]))) # (MeV, MeV/mm)
srim_table=np.array(srim_table)

# Construct the interpolation
srim_interp = interp1d(srim_table[:,0], srim_table[:,1],kind='cubic')
# new_x = np.arange(0.1,50,0.1)
# plt.figure()
# plt.plot(srim_table[:,0], srim_table[:,1], 'o', label='SRIM table')
# plt.plot(new_x, srim_interp(new_x), '-', label='Interpolated -dE/dX')
# plt.xlabel('E (MeV)')
# plt.ylabel('-dE/dx (MeV/mm)')
# plt.title('Stopping power of carbon in carbon')
# plt.legend()

### Step 4
* Construct the yield function.

$$Y_{\rm lab}(\theta_{\rm lab}; E_0)=N_{V}\int_{0}^{E_0}\left[\sigma_{\rm lab}(\theta_{\rm lab}; E)/-\frac{{\rm d} E}{{\rm d} x}\right]{\rm d} E$$

where $\sigma_{\rm lab}(\theta_{\rm lab})=\frac{(1+\gamma^2+2\gamma\cos\theta_{\rm c})^{1.5}}{1+\gamma\cos\theta_{\rm c}}\sigma_{\rm c}(\theta_{\rm c})$

In [ ]:
# Convert COM to Lab
m_a = 12
m_A = 12
m_b = 4
m_B = 20
Q_value=4.62
# gamma = np.sqrt((m_a*m_b)/(m_A*m_B)*(m_b+m_B)/(m_a+m_A)*E_COM[0]/(E_COM[0]+Q))
def yield_angle(theta, angle_index):
    output = azr.extrapolate(theta[:-1],[angle_index])[0]
    energy_c = output[:,0]
    angle_c = output[:,2]
    sigma_c = output[:,3]
    energy_l=(m_a+m_A)/m_A*energy_c
    gamma_local = np.sqrt((m_a*m_b)/(m_A*m_B)*(m_b+m_B)/(m_a+m_A)*energy_c/(energy_c+Q_value))
    sigma_l = (1+gamma_local**2.+2.*gamma_local*np.cos(angle_c/180.*np.pi))**(3./2.)/(1+gamma_local*np.cos(angle_c/180.*np.pi))*sigma_c
    # a_l=np.arccos((gamma_local+np.cos(angle_c/180*np.pi))/np.sqrt(1+gamma_local**2+2*gamma_local*np.cos(angle_c/180*np.pi)))
    sigma_interp = interp1d(energy_l, sigma_l, kind='linear')
    def integrand(E):
        return sigma_interp(E)/srim_interp(E)
    res = np.zeros(num_energies)
    res[0] = quad(integrand, 0.5, energies[0]*m_A/(m_a+m_A))[0] # Integrate from E = 0.5 to 4.4 MeV
    for i in range(num_energies):
        if(i==0):
            continue
        else:
            res[i] = res[i-1]+quad(integrand, energies[i-1]*m_A/(m_a+m_A), energies[i]*m_A/(m_a+m_A))[0] # Integrate from E_{i-1} to E_{i}
    return res * theta[-1]

### Step 5
* Set up a likelihood ($\ln\mathcal{L}$) function.

We'll assume that the data are random draws from uncorrelated Gaussian distributions centered at the $R$-matrix predictions. So, we have
$$
\mathcal{L} = \prod_i \frac{1}{\sqrt{2\pi}\sigma_i} e^{-\frac{1}{2}\left(\frac{y_i-\mu_i}{\sigma_i}\right)^2}~,
$$
where $y_i$ and $\sigma_i$ are the data points and uncertainties, respectively, and $\mu_i$ is the physical model prediction.

We work with $\ln{\mathcal{L}}$ to avoid the numerical difficulties of dealing with extremely small values. The product then becomes a sum,
$$
\ln{\mathcal{L}} = \sum_i \left[-\ln{(\sqrt{2\pi}\sigma_i)} - \frac{1}{2}\left(\frac{y_i-\mu_i}{\sigma_i}\right)^2\right]
$$

In [ ]:
def lnL(theta):
    mu=[]
    for i in range(num_angles):
        mu.append(yield_angle(theta, i))
    mu = np.array(mu, dtype='float32')
    temp_sum = np.zeros(num_energies)
    for i in range(num_energies):
        if(energies[i] == 4.4):
            temp_sum[i]=np.sum(-np.log(np.sqrt(2*np.pi)*yield_err[[8,13,14,16,19,21],i]) - 0.5*((yield_lab[[8,13,14,16,19,21],i] - mu[[8,13,14,16,19,21],i])/yield_err[[8,13,14,16,19,21],i])**2)
        elif(energies[i] == 4.6):
            temp_sum[i]=np.sum(-np.log(np.sqrt(2*np.pi)*yield_err[[1,2,12,14,15,16,19,20,21,22],i]) - 0.5*((yield_lab[[1,2,12,14,15,16,19,20,21,22],i] - mu[[1,2,12,14,15,16,19,20,21,22],i])/yield_err[[1,2,12,14,15,16,19,20,21,22],i])**2)
        elif(energies[i] == 4.7):
            exclude_rows = [9,23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(-np.log(np.sqrt(2*np.pi)*temp_yield_err[:,i]) - 0.5*((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i])**2)
        elif(energies[i] == 5.0):
            exclude_rows = [23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(-np.log(np.sqrt(2*np.pi)*temp_yield_err[:,i]) - 0.5*((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i])**2)
        elif(energies[i] == 5.1):
            exclude_rows = [23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(-np.log(np.sqrt(2*np.pi)*temp_yield_err[:,i]) - 0.5*((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i])**2)
        elif(energies[i] == 5.5):
            exclude_rows = [22,23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(-np.log(np.sqrt(2*np.pi)*temp_yield_err[:,i]) - 0.5*((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i])**2)
        elif(energies[i] == 5.7):
            exclude_rows = [23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(-np.log(np.sqrt(2*np.pi)*temp_yield_err[:,i]) - 0.5*((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i])**2)
        elif(energies[i] == 5.8):
            exclude_rows = [23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(-np.log(np.sqrt(2*np.pi)*temp_yield_err[:,i]) - 0.5*((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i])**2)
        elif(energies[i] == 6.0):
            exclude_rows = [0]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(-np.log(np.sqrt(2*np.pi)*temp_yield_err[:,i]) - 0.5*((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i])**2)
        elif(energies[i] == 6.4):
            exclude_rows = [23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(-np.log(np.sqrt(2*np.pi)*temp_yield_err[:,i]) - 0.5*((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i])**2)
        elif(energies[i] == 6.6):
            exclude_rows = [23]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(-np.log(np.sqrt(2*np.pi)*temp_yield_err[:,i]) - 0.5*((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i])**2)
        elif(energies[i] == 7.4):
            exclude_rows = [0]
            temp_yield_err = np.delete(yield_err, exclude_rows, axis=0)
            temp_yield_lab = np.delete(yield_lab, exclude_rows, axis=0)
            temp_mu = np.delete(mu, exclude_rows, axis=0)
            temp_sum[i]=np.sum(-np.log(np.sqrt(2*np.pi)*temp_yield_err[:,i]) - 0.5*((temp_yield_lab[:,i] - temp_mu[:,i])/temp_yield_err[:,i])**2)
        else:
            temp_sum[i]=np.sum(-np.log(np.sqrt(2*np.pi)*yield_err[:,i]) - 0.5*((yield_lab[:,i] - mu[:,i])/yield_err[:,i])**2)
    return np.sum(temp_sum)

### Step 6
* Construct a posterior ($\ln\mathcal{P}$) function from the likelihood and prior functions.

In [ ]:
def lnP(theta):
    lnpi = lnPi( theta )                        # Calculate the prior
    if not np.isfinite( lnpi ): 
        return -np.inf  # If the prior is not finite, return -inf
    lnl = lnL( theta )                          # Calculate the likelihood
    return lnl + lnpi

### Step 7
* Prepare the MCMC walker and sample the posterior ($\ln\mathcal{P}$) function with emcee.

In [ ]:
nw = 2 * nd  # number of walkers = 2 * number of sampled parameters

# Each walker needs its own starting position. We'll take normally distributed
# random values centered at theta0.
p0 = np.zeros((nw, nd))
# mask = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]) # Widths of the Gaussians can be different.
mask = np.zeros(nd)
for i in range(nd):
    mask[i] = 0.01
for i in range(nw):
    mu = theta0
    sig = np.abs(theta0) * mask
    p0[i, :] = stats.norm(mu, sig).rvs()


# We'll store the chain in chain.h5. (See emcee Backends documentation.)
backend = emcee.backends.HDFBackend("chain.h5")
backend.reset(nw, nd)

nsteps = 4000  # How many saved steps should each walker make?
nthin = 1  # How often should the walkr save a step? n_total = nsteps * nthin
nprocs = nw  # How many Python processes do you want to allocate?

# For a stretch move
move = emcee.moves.StretchMove()

# emcee allows the user to specify the way the ensemble generates proposals.
with Pool(processes=24) as pool:
    sampler = emcee.EnsembleSampler(
        nw, nd, lnP, pool=pool, backend=backend, moves=[move]
    )
    state = sampler.run_mcmc(p0, nsteps, thin_by=nthin, progress=True, tune=True)

#### 1. Posterior function evolution

In [ ]:
# Plot the probability
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

lnp = backend.get_log_prob(discard=100)
ax.plot(lnp)

ax.set_xlabel('Step')
ax.set_ylabel('ln( P )')

plt.show()

#### 2. Parameters evolution

In [ ]:
# Get the samples
nb=3950 # discard the first nb samples
flat_samples = sampler.get_chain(flat=True, discard=nb)

# Plot the parameters
fig, ax = plt.subplots( int(nd / 2)+1, 2, figsize=( 15, 30 ) )

means, lows, highs = [], [], []
final_results = np.zeros(nd)
for i in range( nd ):

        mean = np.percentile( flat_samples[:,i], 50 )
        low = np.percentile( flat_samples[:,i], 16 )
        high = np.percentile( flat_samples[:,i], 84 )
    
        ax[i//2, i%2].set_title( labels[i] )
        ax[i//2, i%2].set_xlabel( "Step" )

        ax[i//2, i%2].plot( flat_samples[:,i], color="tab:blue" )
        ax[i//2, i%2].axhline( mean, color="tab:red", linestyle="-", lw=4 )
        ax[i//2, i%2].axhline( low, color="tab:red", linestyle="--", lw=2 )
        ax[i//2, i%2].axhline( high, color="tab:red", linestyle="--", lw=2 )

        ax[i//2, i%2].set_ylabel( "Value" )

        means.append( mean )
        lows.append( low )
        highs.append( high )

# Save parameters to file
with open( "results/best-emcee.txt", "w" ) as f:
    for i in range( nd ):
        f.write( f"{azr.config.labels[i]}: {means[i]:.5f} + {highs[i]-means[i]:.5f} - {means[i]-lows[i]:.5f}\n" )

#### 3. Corner plot

In [ ]:
# fig = corner.corner(
#     flat_samples, labels=labels, truths=means, quantiles=[0.16, 0.5, 0.84], title_fmt=".4f", fill_contours=True,
#     show_titles=True,
# )

#### 4. Resampling

In [ ]:
def s_factor(theta):
    output = np.array(azr.extrapolate(theta[:-1],[25])[0])
    s_factor_output = np.array([output[:,0], output[:,1], output[:,3], output[:,4]])
    return s_factor_output

def yield_angles(theta):
    mu=[]
    for i in range(num_angles):
        mu.append(yield_angle(theta, i))
    mu = np.array(mu, dtype='float32')
    return mu

def process_parallel():
    # Shuffle and take last 100 samples
    np.random.shuffle(flat_samples)
    samples_to_process = flat_samples[-1000:]
    
    # Create a pool of workers
    with Pool(processes=24) as pool:
        # Map the function to the samples and show progress
        buckets_s_factor = list(tqdm(
            pool.imap(s_factor, samples_to_process),
            total=len(samples_to_process)
        ))
        buckets_yield_angles = list(tqdm(
            pool.imap(yield_angles, samples_to_process),
            total=len(samples_to_process)
        ))
    
    return buckets_s_factor, buckets_yield_angles
buckets_s_factor, buckets_yield_angles = process_parallel()


In [ ]:
# Plot the reampled results and compared it to the data
buckets_s_factor=np.array(buckets_s_factor)

mean = np.percentile(buckets_s_factor, 50, axis=0 ).T
low = np.percentile(buckets_s_factor, 16, axis=0).T
high = np.percentile(buckets_s_factor, 84, axis=0).T

fig, ax = plt.subplots(1,2,figsize=( 15, 5 ))
ax[0].set_xlim(13, 20)
ax[0].set_yscale('log')
ax[0].set_xlabel(r'Excitation energy (MeV, COM)')
ax[0].set_ylabel(r'Cross section (barn)')
ax[0].plot(mean[20:,1], mean[20:,2], label="Extrapolation", color="red")
ax[0].fill_between(mean[20:,1], low[20:,2], high[20:,2], color="red", alpha=0.5 )

ax[1].set_xlim(0, 5)
ax[1].set_yscale('log')
ax[1].set_xlabel(r'Ecm (MeV, COM)')
ax[1].set_ylabel(r'S-factor (MeV b)')
ax[1].plot(mean[:,0], mean[:,3], label="Extrapolation", color="red")
ax[1].fill_between(mean[:,0], low[:,3], high[:,3], color="red", alpha=0.5 )



In [ ]:
buckets_yield_angles=np.array(buckets_yield_angles)

mean = np.percentile(buckets_yield_angles, 50, axis=0 )
low = np.percentile(buckets_yield_angles, 16, axis=0)
high = np.percentile(buckets_yield_angles, 84, axis=0)

fig, ax = plt.subplots( int(num_energies / 2)+1, 2, figsize=( 15, 50 ) )
for i in range( num_energies ):
        ax[i//2, i%2].set_xlabel( "Angle (deg, lab)" )
        ax[i//2, i%2].set_ylabel( "Yield (arb. unit)" )

        mask = yield_err[:, i] > 0
        ax[i//2, i%2].errorbar(angle_lab[mask, i], yield_lab[mask, i], yerr=yield_err[mask, i], linestyle='', capsize=2, fmt="o", label=data_files_list[i])
        ax[i//2, i%2].legend()

        ax[i//2, i%2].plot(angle_lab[mask, i], mean[mask, i], marker="o", label="Extrapolation", color="red")
        ax[i//2, i%2].fill_between(angle_lab[mask, i], low[mask, i], high[mask, i], color="red", alpha=0.5 )


